# Digital I/O with nidaqwrapper

This notebook demonstrates digital input and output using the `DITask` and `DOTask` classes.

Digital I/O in NI-DAQmx operates in two modes:

- **On-demand mode**: Single-sample reads/writes without timing configuration
- **Clocked mode**: Continuous buffered operation with hardware-timed sampling

Both modes support line-level and port-level channel specifications.

## Setup: Mock or Real Hardware

This cell attempts to import nidaqmx. If unavailable, it sets up a lightweight mock so you can explore the API without hardware.

In [ ]:
import numpy as np
from unittest.mock import MagicMock, Mock
import sys

try:
    import nidaqmx
    HW_AVAILABLE = True
    print("✓ nidaqmx available — running with real hardware")
except ImportError:
    HW_AVAILABLE = False
    print("✗ nidaqmx not found — using mock mode")
    
    # Create mock nidaqmx module
    nidaqmx = MagicMock()
    sys.modules['nidaqmx'] = nidaqmx
    
    # Mock system and devices
    mock_system = Mock()
    mock_device = Mock()
    mock_device.name = 'Dev1'
    
    # Mock DI/DO lines for port expansion
    mock_di_lines = []
    mock_do_lines = []
    for i in range(8):
        di_line = Mock()
        di_line.name = f'Dev1/port0/line{i}'
        mock_di_lines.append(di_line)
        
        do_line = Mock()
        do_line.name = f'Dev1/port1/line{i}'
        mock_do_lines.append(do_line)
    
    mock_device.di_lines = mock_di_lines
    mock_device.do_lines = mock_do_lines
    
    mock_system.devices = [mock_device]
    mock_system.tasks.task_names = []
    nidaqmx.system.System.local.return_value = mock_system
    
    # Mock Task class
    class MockTask:
        def __init__(self, new_task_name=''):
            self.name = new_task_name
            self.di_channels = Mock()
            self.do_channels = Mock()
            self.timing = Mock()
            self._channels = []
            
            # Mock add_di_chan to track channels
            def add_di_chan(lines, name_to_assign_to_lines='', line_grouping=None):
                ch = Mock()
                ch.name = name_to_assign_to_lines
                ch.physical_channel = Mock()
                ch.physical_channel.name = lines
                self._channels.append(ch)
            
            # Mock add_do_chan to track channels
            def add_do_chan(lines, name_to_assign_to_lines='', line_grouping=None):
                ch = Mock()
                ch.name = name_to_assign_to_lines
                ch.physical_channel = Mock()
                ch.physical_channel.name = lines
                self._channels.append(ch)
            
            self.di_channels.add_di_chan = add_di_chan
            self.do_channels.add_do_chan = add_do_chan
            self.di_channels.__iter__ = lambda s: iter([ch for ch in self._channels])
            self.do_channels.__iter__ = lambda s: iter([ch for ch in self._channels])
            self.di_channels.__len__ = lambda s: len(self._channels)
            self.do_channels.__len__ = lambda s: len(self._channels)
            
        @property
        def channel_names(self):
            return [ch.name for ch in self._channels]
        
        def read(self, number_of_samples_per_channel=1):
            # Return fake digital data
            n_channels = len(self._channels)
            if number_of_samples_per_channel == -1:  # READ_ALL_AVAILABLE
                n_samples = 10
                if n_channels == 1:
                    return [bool(i % 2) for i in range(n_samples)]
                else:
                    return [[bool((i+ch) % 2) for i in range(n_samples)] for ch in range(n_channels)]
            else:
                if n_channels == 1:
                    return True
                else:
                    return [bool(ch % 2) for ch in range(n_channels)]
        
        def write(self, data, auto_start=False):
            pass  # Accept data without error
        
        def start(self):
            pass
        
        def close(self):
            pass
    
    nidaqmx.task.Task = MockTask
    
    # Mock constants
    nidaqmx.constants.LineGrouping.CHAN_PER_LINE = 'chan_per_line'
    nidaqmx.constants.AcquisitionType.CONTINUOUS = 'continuous'
    nidaqmx.constants.READ_ALL_AVAILABLE = -1

# Now import nidaqwrapper
from nidaqwrapper import DITask, DOTask

## Digital Input: On-Demand Mode

On-demand mode reads single samples without timing configuration. This is ideal for reading switch states, button presses, or other slowly-changing digital signals.

Key points:
- Create `DITask` without a `sample_rate` parameter
- Call `add_channel()` to specify which lines to read
- Call `start()` to prepare the task (no timing config needed)
- Use `read()` to get current state of all lines

In [ ]:
# On-demand digital input example
di = DITask(task_name='switches')
print(f"Mode: {di.mode}")  # 'on_demand'

# Add channels (lines can be individual, range, or port)
di.add_channel('switch_inputs', lines='Dev1/port0/line0:3')
print(f"Channels: {di.channel_list}")
print(f"Number of channels: {di.number_of_ch}")

# Start task (no timing config for on-demand)
di.start()

# Read current state
data = di.read()
print(f"\nSingle sample read: {data}")
print(f"Shape: {data.shape}, dtype: {data.dtype}")

# Clean up
di.clear_task()

## Digital Input: Clocked Mode

Clocked mode reads buffered samples at a specified rate. This is required for fast digital capture or synchronized multi-channel acquisition.

Key points:
- Create `DITask` with a `sample_rate` parameter
- Call `start(start_task=True)` to configure timing and begin acquisition
- Use `read_all_available()` to retrieve buffered samples
- Data shape is `(n_samples, n_lines)`

In [ ]:
# Clocked digital input example
di = DITask(task_name='fast_di', sample_rate=1000.0)
print(f"Mode: {di.mode}")  # 'clocked'

# Add channels
di.add_channel('encoder_signals', lines='Dev1/port0/line0:7')

# Start with timing configuration
di.start(start_task=True)

# Read buffered data
data = di.read_all_available()
print(f"\nBuffered read: {data.shape} samples")
print(f"First 5 samples:\n{data[:5]}")

# Clean up
di.clear_task()

## Digital Input: Port Expansion

When you specify a port without line numbers (e.g., `'Dev1/port0'`), the wrapper automatically expands it to the full line range (e.g., `'Dev1/port0/line0:7'` for an 8-line port).

This makes configuration more concise when reading entire ports.

In [ ]:
# Port expansion example
di = DITask(task_name='full_port')

# This automatically expands to 'Dev1/port0/line0:7' (8 lines)
di.add_channel('port0_all', lines='Dev1/port0')

# Verify expansion worked
print(f"Channels: {di.channel_list}")
print(f"Number of channels: {di.number_of_ch}")  # Should be 8 for typical port

di.clear_task()

## Digital Output: On-Demand Mode

On-demand mode writes single samples to output lines. This is ideal for controlling LEDs, relays, or other digital actuators.

Key points:
- Create `DOTask` without a `sample_rate` parameter
- Call `add_channel()` to specify which lines to control
- Call `start()` to prepare the task
- Use `write()` with bool, int, list, or array data

In [ ]:
# On-demand digital output example
do = DOTask(task_name='leds')
print(f"Mode: {do.mode}")  # 'on_demand'

# Add channels
do.add_channel('led_array', lines='Dev1/port1/line0:3')
print(f"Channels: {do.channel_list}")

# Start task
do.start()

# Write different data types
do.write([True, False, True, False])  # List of bools
print("Wrote list of bools")

do.write(np.array([1, 0, 1, 0]))  # Array of ints (converted to bool)
print("Wrote array of ints")

do.write(np.array([False, True, False, True]))  # Array of bools
print("Wrote array of bools")

# Clean up
do.clear_task()

## Digital Output: Clocked Mode

Clocked mode generates buffered waveforms at a specified rate. This is used for pattern generation, PWM signals, or synchronized multi-line output.

Key points:
- Create `DOTask` with a `sample_rate` parameter
- Call `start(start_task=True)` to configure timing and begin output
- Use `write_continuous()` with shape `(n_samples, n_lines)`
- Data must be boolean type

In [ ]:
# Clocked digital output example
do = DOTask(task_name='pattern_gen', sample_rate=1000.0)
print(f"Mode: {do.mode}")  # 'clocked'

# Add channels
do.add_channel('output_lines', lines='Dev1/port1/line0:3')

# Start with timing configuration
do.start(start_task=True)

# Generate pattern data
n_samples = 100
n_lines = 4
pattern = np.zeros((n_samples, n_lines), dtype=bool)

# Create square waves at different frequencies
for line in range(n_lines):
    period = 10 * (line + 1)  # Different period for each line
    pattern[:, line] = (np.arange(n_samples) % period) < (period // 2)

print(f"\nPattern shape: {pattern.shape}")
print(f"First 10 samples:\n{pattern[:10]}")

# Write buffered data
do.write_continuous(pattern)
print("\nWrote continuous pattern")

# Clean up
do.clear_task()

## Context Manager Usage

Both `DITask` and `DOTask` support context managers for automatic cleanup. This is the recommended pattern for production code.

In [ ]:
# Digital input with context manager
with DITask(task_name='ctx_di', sample_rate=1000) as di:
    di.add_channel('inputs', lines='Dev1/port0/line0:3')
    di.start(start_task=True)
    data = di.read_all_available()
    print(f"DI: Read {data.shape[0]} samples from {data.shape[1]} lines")
# Task automatically cleaned up

# Digital output with context manager
with DOTask(task_name='ctx_do') as do:
    do.add_channel('outputs', lines='Dev1/port1/line0:3')
    do.start()
    do.write([True, False, True, False])
    print("DO: Wrote single sample")
# Task automatically cleaned up

## Configuration Persistence

Both classes support saving/loading configurations as TOML files. This enables hardware-portable test setups.

In [ ]:
import tempfile
import os

# Create and save DI configuration
di = DITask(task_name='config_di', sample_rate=1000)
di.add_channel('signals', lines='Dev1/port0/line0:7')

with tempfile.TemporaryDirectory() as tmpdir:
    di_path = os.path.join(tmpdir, 'di_config.toml')
    di.save_config(di_path)
    
    # Show saved config
    with open(di_path, 'r') as f:
        print("DI Configuration:")
        print(f.read())
    
    # Load it back
    di_loaded = DITask.from_config(di_path)
    print(f"\nLoaded task: {di_loaded.task_name}")
    print(f"Sample rate: {di_loaded.sample_rate} Hz")
    print(f"Channels: {di_loaded.channel_list}")
    di_loaded.clear_task()

di.clear_task()

# Create and save DO configuration
do = DOTask(task_name='config_do')
do.add_channel('outputs', lines='Dev1/port1')

with tempfile.TemporaryDirectory() as tmpdir:
    do_path = os.path.join(tmpdir, 'do_config.toml')
    do.save_config(do_path)
    
    # Show saved config
    with open(do_path, 'r') as f:
        print("\nDO Configuration:")
        print(f.read())
    
    # Load it back
    do_loaded = DOTask.from_config(do_path)
    print(f"\nLoaded task: {do_loaded.task_name}")
    print(f"Mode: {do_loaded.mode}")
    print(f"Channels: {do_loaded.channel_list}")
    do_loaded.clear_task()

do.clear_task()

## Advanced: Raw Task Injection

For advanced users who need features not exposed by the wrapper API, you can create a raw `nidaqmx.Task` and wrap it using `from_task()`.

Note: This only works with real hardware (not in mock mode).

In [ ]:
if HW_AVAILABLE:
    # Create raw nidaqmx task with custom configuration
    raw_task = nidaqmx.Task()
    raw_task.di_channels.add_di_chan(
        "Dev1/port0/line0:3",
        line_grouping=nidaqmx.constants.LineGrouping.CHAN_PER_LINE
    )
    raw_task.timing.cfg_samp_clk_timing(rate=1000)
    
    # Wrap in DITask
    di = DITask.from_task(raw_task)
    print(f"Wrapped task: {di.task_name}")
    print(f"Mode: {di.mode}")
    print(f"Channels: {di.channel_list}")
    
    # Use wrapper read methods
    raw_task.start()
    data = di.read_all_available()
    print(f"Read {data.shape[0]} samples")
    
    # Caller must close the task
    raw_task.close()
else:
    print("Raw task injection requires real hardware")

## Summary

This notebook demonstrated:

1. **DITask** for digital input:
   - On-demand mode with `read()`
   - Clocked mode with `read_all_available()`
   - Port expansion for full-port specifications

2. **DOTask** for digital output:
   - On-demand mode with `write()`
   - Clocked mode with `write_continuous()`
   - Flexible data input (bool, int, list, array)

3. **Best practices**:
   - Context managers for automatic cleanup
   - TOML configuration for hardware portability
   - Raw task injection for advanced features

Both classes use `CHAN_PER_LINE` grouping and follow the direct delegation pattern where the underlying `nidaqmx.Task` is the single source of truth.